In [ ]:
#@title
###########################################################################
#
#  Copyright 2021 Google Inc.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This solution, including any related sample code or data, is made available 
# on an “as is,” “as available,” and “with all faults” basis, solely for 
# illustrative purposes, and without warranty or representation of any kind. 
# This solution is experimental, unsupported and provided solely for your 
# convenience. Your use of it is subject to your agreements with Google, as 
# applicable, and may constitute a beta feature as defined under those 
# agreements.  To the extent that you make any data available to Google in 
# connection with your use of the solution, you represent and warrant that you 
# have all necessary and appropriate rights, consents and permissions to permit 
# Google to use and process that data.  By using any portion of this solution, 
# you acknowledge, assume and accept all risks, known and unknown, associated 
# with its usage, including with respect to your deployment of any portion of 
# this solution in your systems, or usage in connection with your business, 
# if at all.
###########################################################################

# Imports, Globals, and Helper Functions

In [ ]:
#@title
!pip install boruta | grep -v 'already satisfied'
!pip install relativeImp | grep -v 'already satisfied'
from relativeImp import relativeImp
from sklearn.ensemble import RandomForestRegressor
from google.colab import files
from scipy.optimize import minimize, Bounds, NonlinearConstraint
from google.cloud import bigquery
from google.cloud.bigquery import magics
from google.colab import auth
from sklearn import preprocessing
from IPython.display import Javascript
import os
import re
import time
import random
import io
import sys
import json
import pandas as pd
import numpy as np
import ipywidgets as widgets
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

TARGET_VAR = None
TARGET_VARIABLE = "TARGET_VARIABLE"

def current_milli_time():
    return round(time.time() * 1000)
timer = {}
def startTimer(line):
  global timer
  if line in timer.keys():
    timer[line]['start'] = current_milli_time()
  else:
    timer[line] = {}
    timer[line]['total'] = 0
    timer[line]['start'] = current_milli_time()

def endTimer(line):
  global timer
  timer[line]['total'] = timer[line]['total'] + (current_milli_time() - timer[line]['start'])

#This function creates every combination of Lag, Decay (carryover), and Curve Shape
def Transformation(dataframe, x, lagVal, alphaVal, decayVal):
    lag = []
    data = dataframe[x].shift(int(lagVal)).to_frame()
    data.columns = [col_name+'Lag'+str(lagVal)for col_name in data.columns]
    lag.append(data)
    # see pd.concat documentation for more info
    lag = pd.concat(lag,axis=1)
    lag=lag.fillna(0)
    Alpha = []
    
    data = pow(lag,alphaVal)
    data.columns = [col_name+'Alpha'+str(alphaVal)for col_name in data.columns]
    # store DataFrame in list
    Alpha.append(data)
    # see pd.concat documentation for more info
    Alpha = pd.concat(Alpha,axis=1) 
    Decay=[]
    
    data = Alpha.copy()
    data.columns = [col_name+'Decay'+str(decayVal)for col_name in data.columns]
    for j in range(0, len(Alpha.columns)):
      data[data.columns[j]] = buildAlphaValues(len(data[data.columns[j]])-1, data[data.columns[j]], decayVal)
    Decay.append(data)
    Decay = pd.concat(Decay,axis=1)
    
    return Decay

def createTransformations(df1, df2, categoryMapping):
  columns = df2.columns
  sales = df1[[TARGET_VAR]] 
  sys.setrecursionlimit(len(sales.index)+100)

  all_data = [] 
  for col in columns:
    #If the data is weekly instead of daily, update the below variable to False
    newdf = Transformation(df2, col, parseLag(categoryMapping[stripQualifiers(col, False)]['fullColumn']), parseAlpha(categoryMapping[stripQualifiers(col, False)]['fullColumn']), parseDecay(categoryMapping[stripQualifiers(col, False)]['fullColumn']))
    corr_df = pd.concat([sales, newdf], axis=1)
    corr = corr_df.corr().sort_values(TARGET_VAR, ascending=False)
    new_vals= corr.iloc[1:4 , 0:1].index.tolist()
    data = newdf[new_vals]
    all_data.append(data)
  final_data = pd.concat(all_data,axis=1)
  
  return final_data

def buildAlphaValues(index, original_column, percent):
  if index == 0:
    return [original_column[0] * percent]
  else:
    previous_values = buildAlphaValues(index-1, original_column, percent)
    previous_values.append(original_column[index] * percent + previous_values[index-1] * (1-percent))
    return previous_values

def parseLag(column):
  if 'Lag' not in column:
    return 0.0
  return float(column.split('Lag')[len(column.split('Lag'))-1].split('Alpha')[len(column.split('Lag')[len(column.split('Lag'))-1].split('Alpha'))-2].replace('_', '.'))

def parseAlpha(column):
  if 'Alpha' not in column:
    return 1.0
  return float(column.split('Alpha')[len(column.split('Alpha'))-1].split('Decay')[len(column.split('Alpha')[len(column.split('Alpha'))-1].split('Decay'))-2].replace('_', '.'))

def parseDecay(column):
  if 'Decay' not in column:
    return 1.0
  return float(column.split('Decay')[len(column.split('Decay'))-1].replace('_', '.'))

def stripQualifiers(column, leading=True):
  retString = column
  if retString.startswith('cost_') and leading:
    retString = retString.split('cost_')[1]
  if retString.startswith('Paid_') and leading:
    retString = retString.split('Paid_')[1]
  if "Lag" in retString:
    retString = retString.split('Lag')[len(column.split('Lag'))-2]
  return retString

final_iteration_revenue_data = []
def minimize_function(x, original_data, chosen_columns, pointEstimates):
  final_revenue_data = []
  for attribute_idx, col in enumerate(chosen_columns):
    # idx is essentially the attribute we are using, order must be the same between x and columns of df
    lag = parseLag(col)
    alpha = parseAlpha(col)
    decay = parseDecay(col)
    
    #Step 1: Optimization
    #Opt.         = (original data X current opt spend) / sum original spend
    optimized_data = (original_data[stripQualifiers(col, False)] * x[attribute_idx]) / original_data[stripQualifiers(col, False)].sum()

    #Step 2: Lag
    lagged_data = []
    for index, optimize_value in enumerate(optimized_data):
      if index < lag:
        lagged_data.append(0)
      else:
        lagged_data.append(optimized_data[index - int(lag)])

    #Step 3: Alpha
    alpha_data = []
    for lagged_value in lagged_data:
      alpha_data.append(lagged_value ** alpha)

    #Step 4: Decay
    decay_data = []
    decay_n = 0
    for alpha_value in alpha_data:
      decay_n = (alpha_value * decay) + (decay_n * (1.0-decay))
      decay_data.append(decay_n)

    #Step 5: Coeffs
    final_data = []
    for i in decay_data:
      final_data.append(i * pointEstimates[attribute_idx])

    #Step 6: Sum to revenue
    final_revenue_data.append(sum(final_data))

  #Set the final calculations for display later
  global final_iteration_revenue_data
  final_iteration_revenue_data = final_revenue_data
  
  return (sum(final_revenue_data) / sum(x)) * -1

def createBounds(mins, maxes):
  return Bounds(lb=mins, ub=maxes)

def constraintSum(x):
  return [sum(x)]

def createConstraints(budget):
  return NonlinearConstraint(fun=constraintSum, lb=[budget], ub=[budget])

def optimize(original_data, chosen_columns, pointEstimates, mins, maxes, budget):
  return minimize(fun=minimize_function, x0=mins, args=(original_data, chosen_columns, pointEstimates), method='SLSQP', bounds=createBounds(mins, maxes), constraints=createConstraints(budget))

# Import Data
Only one of the following sections should be executed, either import from CSV, BigQuery, or Google Sheets.

## Upload Original Data CSV (Option #1)

In [ ]:
#@title
original_file = files.upload()
original_data_import = pd.read_csv(io.BytesIO(original_file[list(original_file.keys())[0]])).fillna(0)

## Import Original Data from BigQuery (Option #2)

In [ ]:
#@title
auth.authenticate_user()
bigquery.USE_LEGACY_SQL = False

magics.context.project = '' #***Update with BigQuery Project name***
client = bigquery.Client(project=magics.context.project)
%load_ext google.cloud.bigquery
#***Update with BigQuery Project name*** 
project_id = '' #@param {type:"string"}
table_id = '' #@param {type:"string"}
query = 'select * from `' + table_id + '`'
#***Update with BigQuery Project name***
original_data_import = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

## Import Data from Sheets (Option #3)

In [ ]:
auth.authenticate_user()
sheet_name = 'original_data2' #@param {type:"string"}
worksheet = gc.open(sheet_name).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
original_data_import = pd.DataFrame.from_records(rows)
original_data_import = original_data_import.rename(columns=original_data_import.iloc[0]).drop(original_data_import.index[0]).reset_index(drop=True).astype('float')

# Import Model Column Names

Please include target variable & all features in final model. This will inform the feature level transformations for the optimization model.

Feature names should have Lag, Alpha, and Decay values. (i.e. Social_Twitter_RemarketingLag13Alpha0.6Decay0.6)

**Do not re-run the sell after you input feature names**

In [ ]:
#@title
columnList = widgets.Text(layout=widgets.Layout(width='500px'))
print("Comma separated list of feature names")
columnList

# Data Categorization
Run the cell below and then enter a comma separated list of channel names if you wish to group tactics by channel. [This is optional and can be left blank.]

i.e. Display, Social, Search,...

**Do not re-run the sell after you input channel names**


In [ ]:
#@title
categoryList = widgets.Text(layout=widgets.Layout(width='500px'))
print("Comma separated list of optional channel names to organize your data")
categoryList

Now, run the cell below. Here you'll classify the target variable, non-paid media, and the channel (if channel was provided above). 

Target variable: in most cases this is your revenue column.

Note: If channel names were not provided above, and the feature is not the target or non-paid media, you can leave blank.

**Do not re-run the sell after you classify the features**

In [ ]:
#@title

categories = categoryList.value.split(',')
modelColumns = columnList.value.split(',')
categories[:] = [c.strip() for c in categories]
modelColumns[:] = [col.strip().replace('\'', '').replace('\"', '') for col in modelColumns]
categories.append("TARGET_VARIABLE")
categories.append("Non-Paid Media")
categories.append("Search")
categoryDropdowns = []
categoryMapping = {}
for col in list(modelColumns):
  dropdown = widgets.Dropdown(options=categories)
  categoryMapping[stripQualifiers(col, False)] = {'dropdown': dropdown, 'fullColumn': col}
  categoryDropdowns.append(widgets.HBox([widgets.Label(stripQualifiers(col), layout=widgets.Layout(width='300px')), dropdown]))
widgets.VBox(categoryDropdowns)

#Prep Data for Optimization
#### (Create Transformations, Standardization, Calculate Point Estimates, Optimization Functions)

In [ ]:
#@title
targetVarsFound = 0
toBeDropped = []
for col in list(modelColumns):
  if categoryMapping[stripQualifiers(col, False)]['dropdown'].value == TARGET_VARIABLE:
    TARGET_VAR = col
    toBeDropped.append(TARGET_VAR)
    targetVarsFound += 1


if targetVarsFound == 1:
  #Variables that do not need to be transformed (i.e. lagged, carryover, diminishing returns)
  # Don't transform your y var, any flag variables, or variables that should be fully recognized on that specific day
  original_data_target_var = original_data_import[[
  TARGET_VAR, 
  ]]

  #data that needs to be transformed
  trimmed_original_data = original_data_import.drop(toBeDropped, axis=1)

  #This step can take several minutes
  #This creates all combinations and then calculates the correlation between each variable and the Y variable. 
  #Returns the top 3 highest correlated features
  
  transformed_original_data = createTransformations(original_data_target_var, trimmed_original_data, categoryMapping)
  feature_selected_transformed_w_target = pd.concat([original_data_target_var, transformed_original_data], axis=1)
  # trim the dataset in accordance with the max lag
  maxLag = 0
  for col in list(feature_selected_transformed_w_target):
    lag = parseLag(col)
    maxLag = lag if lag > maxLag else maxLag
  feature_selected_transformed_w_target = feature_selected_transformed_w_target.iloc[int(maxLag):, :]
  feature_selected_transformed_w_target = feature_selected_transformed_w_target.reset_index(drop=True)
  feature_selected_transformed_w_o_target = feature_selected_transformed_w_target.drop([TARGET_VAR], axis=1)
  original_data = original_data_import.iloc[int(maxLag):, :]
  original_data = original_data.reset_index(drop=True)

  # separate the independent and dependent variables
  stand_x = feature_selected_transformed_w_target.loc[:, ~feature_selected_transformed_w_target.columns.isin([TARGET_VAR])] 
  stand_y = feature_selected_transformed_w_target[[TARGET_VAR]] 

  # standardization of dependent variables
  standard = preprocessing.scale(stand_x)

  #converting standardized data back to a datframe
  feature_selected_transformed_w_o_target_standardized = pd.DataFrame(standard, columns = feature_selected_transformed_w_target.loc[:, ~feature_selected_transformed_w_target.columns.isin([TARGET_VAR])].columns)

  #x variables standardized with mean 0 and standard dev of 1
  feature_selected_transformed_w_target_standardized = pd.concat([feature_selected_transformed_w_target[[TARGET_VAR]], feature_selected_transformed_w_o_target_standardized], axis=1)

  #Calculate relative importance for features vs. impressions
  relativeImpDf = relativeImp(feature_selected_transformed_w_target_standardized, outcomeName = TARGET_VAR, driverNames = list(feature_selected_transformed_w_o_target_standardized))

  #Calculate coefficients
  impByDriver = relativeImpDf.set_index('driver')
  impByDriver['driverTrimmed'] = [stripQualifiers(i, False) for i in list(impByDriver.index.values)]
  impByDriver = impByDriver.set_index('driverTrimmed')
  impByDriver['revByTactic'] = impByDriver['rawRelaImpt'] * feature_selected_transformed_w_target[TARGET_VAR].sum()
  impByDriver['pointEstimate'] = np.zeros(len(impByDriver))
  for col in list(feature_selected_transformed_w_o_target):
    impByDriver['pointEstimate'][stripQualifiers(col, False)] = impByDriver['revByTactic'][stripQualifiers(col, False)] / original_data[stripQualifiers(col, False)].sum()
  impByDriver = impByDriver.sort_values(by=['pointEstimate'])
  print("Recommendations:")
  if len(impByDriver.index) > 5:
    print("Increase spend for: " + impByDriver.index[-1] + ", " + impByDriver.index[-2] + ", " + impByDriver.index[-3])
  else:
    print("Increase spend for: " + impByDriver.index[-1])

  if len(impByDriver.index) > 3:
    if impByDriver['pointEstimate'][impByDriver.index[2]] < 1.0:
      print("Decrease spend for: " + impByDriver.index[0] + ", " + impByDriver.index[1] + ", " + impByDriver.index[2])
    else:
      print("Decrease spend for: " + impByDriver.index[0] + ", " + impByDriver.index[1])
  else:
    print("Decrease spend for: " + impByDriver.index[0])
else:
  print("Please ensure that one and only one TARGET_VARIABLE is chosen above.")



# **Optimizer RBA Tool**
 
## Instructions
 
Run the cell to dynamically generate the UI based on categorizations set above.
 
1.  Enter the total budget you wish to optimize.
2.  Enter min & max budget by tactic.
* Use all available information  to help inform the min & max (i.e. Last touch model, in platform attribution, ad hoc analyses, etc.)
* Pro-tip: press +/-20% to autofill min & max proportionally to original budgets
3.  Once you're comfortable with the min & max for each tactic, press "Do it!" to optimize budget.
 
## Additional Details
 
Based on the amount of data, the optimization may take a while. Interrupting the calculation may lead to an unresponsive runtime. If possible, please wait until the optimization finishes to make any changes.
 
## FAQ
 
What is the model optimizing for?
*  ROI. Baseline ROI is calculated with the original dataset. The optimization model is optimizing the overall budget in order to increase ROI.

How do we mitigate the concern that certain tactics have caps based on frequency/reach (i.e. Search, Retargeting, etc.)
*  This can be accounted for in the max spend for that tactic

Confirm that this tool can not be used if impressions or clicks are modeled (unless backing into spend)
*  Correct, for this tool to work correctly, spend must be used for the tactic level input.



In [ ]:
#@title



#Layout and Styles for the buttons, inputs and loaders
buttonLayout = widgets.Layout(margin="10px 10px", width='300px')
pButtonLayout = widgets.Layout(margin="-25px 2px 10px 2px", width='300px')
button = widgets.Button(description="Do it!", layout=buttonLayout)
p20button = widgets.Button(description="+/- 20%", layout=pButtonLayout)
tableLayout = widgets.Layout(width='300px', display="flex", justify_content="center")
inputLayout = widgets.Layout(width='300px', display="flex", justify_content="center")
headerLayoutNoBottomMargin = widgets.Layout(width='300px', display="flex", justify_content="center")
headerLayout = widgets.Layout(width='300px', display="flex", justify_content="center", margin="0px 0px 50px")
warningLayout = widgets.Layout(width='300px', display="flex", justify_content="center")
html_styles = '''
    <style>
      .warning_1 input { 
          color:red !important; 
      }
      .warning_2 input { 
          color:red !important; 
      }
      .warning_3 input { 
          color:red !important; 
      }
      .warning_4 input { 
          color:red !important; 
      }
      .lds-dual-ring {
        display: inline-block;
        width: 30px;
        height: 30px;
        margin-bottom: 8px;
        margin-left: 20px;
        margin-top: 10px
      }
      .lds-dual-ring:after {
        content: " ";
        display: block;
        width: 16px;
        height: 16px;
        margin: 2px;
        border-radius: 50%;
        border: 3px solid #fff;
        border-color: #fff transparent #fff transparent;
        animation: lds-dual-ring 1.2s linear infinite;
      }
      @keyframes lds-dual-ring {
        0% {
          transform: rotate(0deg);
        }
        100% {
          transform: rotate(360deg);
        }
      }
      .widget-label {
        min-width:300px !important
      }
      .widget-text {
        min-width:300px !important
      }
      input {
        min-width:300px !important
      }
      .overflow_style {
        min-width:2400px !important
      }
    </style>
'''
html_loader = '''
  <div></div>
'''

# Update this flag to prepopulate optimizer with test data
useTestData = False
testData = {
    'email_clicked': [0,0],
    'affiliate_cost': 	 [154029, 	 231044], 
  'sitedirect_Brand': 	 [218788, 	 328182], 
  'sitedirect_eCom': 	 [10894, 	 16341], 
  'Display_DBM_Brand_Prospecting': 	 [44261, 	 66392], 
  'Display_DBM_eCom_Prospecting': 	 [31821, 	 47731], 
  'Display_DBM_eCom_Remarketing': 	 [69375, 	 104063], 
  'Display_GDA_eCom_Remarketing': 	 [12622, 	 18933], 
  'Display_Xandr_Brand_Prospecting': 	 [40630, 	 60945], 
  'Social_Facebook_Brand_Prospecting': 	 [26931, 	 40397], 
  'Social_Facebook_eCom_Prospecting': 	 [23162, 	 34743],
  'Social_Facebook_eCom_Remarketing': [0,0], 
  'Social_FBIG_Brand_Prospecting': 	 [18128, 	 27192], 
  'Social_FBIG_eCom_Prospecting': 	 [63486, 	 95229], 
  'Social_FBIG_eCom_Remarketing': 	 [324853, 	 487280], 
  'Social_Instagram_Brand_Prospecting': 	 [45641, 	 68462], 
  'Social_Instagram_eCom_Prospecting': 	 [4516, 	 6774], 
  'Social_Instagram_eCom_Remarketing': 	 [1380, 	 2070], 
  'Social_Pinterest_Brand_Prospecting': [15602, 	 23402], 
  'Social_Pinterest_eCom_Prospecting': 	 [4935,	 7402], 
  'Social_Pinterest_eCom_Remarketing': 	 [29267, 	 43900], 
  'Social_Snapchat_Brand_Prospecting': 	 [34373, 	 51559], 
  'Social_Snapchat_eCom_Prospecting':	 [5174, 	 7761], 
  'Social_Snapchat_eCom_Remarketing': 	 [11051, 	 16576], 
  'Social_TikTok_Brand_Prospecting': 	 [1594, 	 2392], 
  'Social_TikTok_Brand_Remarketing': 	 [2818, 	 4228], 
  'Social_TikTok_eCom_Remarketing': 	 [961, 	 1441], 
  'Social_Twitter_eCom_Prospecting':	 [2621, 	 3931], 
  'Social_Twitter_eCom_Remarketing': 	 [1087, 	 1630],
  'Social_Twitter_Brand_Prospecting': [0,0],
  'search_sa': [0,0]
}
rows = []
displayUI = []
metrics = {}

#Error handling is a little convoluted here, but it is easier to manage error classes across rows and columns when they map 1:1
errorMessage1 = "Attribute minimums exceed the category total minimum"
warning1 = "warning_1"
errorMessage2 = "Attribute maximums exceed the category total maximum"
warning2 = "warning_2"
errorMessage3 = "Attribute minimum exceeds attribute maximum"
warning3 = "warning_3"
errorMessage4 = "Category minimum exceeds category maximum"
warning4 = "warning_4"
errorMessage5 = "Consider increasing Search minimum to be within 20% of current budget"
warning5 = "warning_5"

#Number formatting to add comma every 3 digits
def formatNumber(number):
  return "{:,}".format(number)

def budgetChangeHandler(change):
  mins = []
  maxes = []
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        mins.append(attr['minSpend'].value)
        maxes.append(attr['maxSpend'].value)
  if metrics['budget'].value > sum(maxes) or metrics['budget'].value < sum(mins):
    printToOutputWidget(metrics['status'], "Your budget lies outside the min/max bounds for your table")
  else:
    printToOutputWidget(metrics['status'], "")

#Change handler fired every time input is changed. Handles error checking and category level updates
def attributeChangeHandler(change):
  for idx, attr in enumerate(rows):
    if attr['isCategoryRow']:
      attr['minSpend'].value = fetchTotalForCategory(attr['category'], True)
      attr['maxSpend'].value = fetchTotalForCategory(attr['category'], False)

    isMin = change.owner == attr['minSpend']
    if change.owner == attr['maxSpend'] or isMin:
      if fetchTotalForCategory(attr['category'], isMin) > fetchCategoryValue(attr['category'], isMin):
        if attr['isCategoryRow']:
          clearCategoryWarning(attr['category'], errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)
        setWarning(idx, errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)
      else:
        if attr['isCategoryRow']:
          clearWarning(idx, errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)
        clearCategoryWarning(attr['category'], errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)
      
      if attr['minSpend'].value > attr['maxSpend'].value:
        if attr['isCategoryRow']:
          clearWarning(idx, errorMessage4, warning4, not isMin)
          setWarning(idx, errorMessage4, warning4, isMin)
        else:
          clearWarning(idx, errorMessage3, warning3, not isMin)
          setWarning(idx, errorMessage3, warning3, isMin)
      else:
        if attr['isCategoryRow']:
          clearWarning(idx, errorMessage4, warning4, isMin)
          clearWarning(idx, errorMessage4, warning4, not isMin)
        else:
          clearWarning(idx, errorMessage3, warning3, isMin)
          clearWarning(idx, errorMessage3, warning3, not isMin)

    if attr['category'] is 'Search' and not attr['isCategoryRow']:
      total = 0
      for idx2, attr2 in enumerate(rows):
        if not attr2['isCategoryRow'] and attr2['category'] is not 'Total':
          total = total + original_data[attr2['attr']].sum()

      rowTotal = original_data[attr['attr']].sum()
      # ~78% to allow for rounding room in the initial 20% calculations
      p80 = (rowTotal/total) * metrics['budget'].value * 0.78
      if attr['minSpend'].value < p80:
        setWarning(idx, errorMessage5, warning5, isMin)
      else:
        clearWarning(idx, errorMessage5, warning5, isMin)

  mins = []
  maxes = []
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        mins.append(attr['minSpend'].value)
        maxes.append(attr['maxSpend'].value)
  if metrics['budget'].value > sum(maxes) or metrics['budget'].value < sum(mins):
    printToOutputWidget(metrics['status'], "Your budget lies outside the min/max bounds for your table")
  else:
    printToOutputWidget(metrics['status'], "")

#Helper function for error handling
def setWarning(rowIndex, message, warning, isMin):
  rows[rowIndex]['minSpend' if isMin else 'maxSpend'].add_class(warning)
  rows[rowIndex]['errors'].add(message)
  printToOutputWidget(rows[rowIndex]['warning'], message)

#Helper function for row level error handling
def clearWarning(rowIndex, message, warning, isMin):
  out = rows[rowIndex]['warning']
  rows[rowIndex]['minSpend' if isMin else 'maxSpend'].remove_class(warning)
  with out:
    if len(rows[rowIndex]['errors']) != 0:
      rows[rowIndex]['errors'].discard(message)
      out.clear_output()
      queueLength = len(rows[rowIndex]['errors'])
      if queueLength != 0:
        print(next(iter(rows[rowIndex]['errors'])))

#Helper function for category level error handling
def clearCategoryWarning(category, message, warning, isMin):
  for idx, attr in enumerate(rows):
    if attr['category'] == category:
      clearWarning(idx, message, warning, isMin)

#Getter function for category
def fetchCategoryValue(category, isMin):
  for idx, attr in enumerate(rows):
    if attr['category'] == category and attr['isCategoryRow']:
      return attr['minSpend' if isMin else 'maxSpend'].value

#Helper function to sum the values for a category
def fetchTotalForCategory(category, isMinTotal):
  total = 0
  for idx, attr in enumerate(rows):
    if attr['category'] == category and not attr['isCategoryRow']:
      total = total + (attr['minSpend'].value if isMinTotal else attr['maxSpend'].value)
  return total

#Generate a row for the output grid. Category rows have empty attribute values. This also populates
# the test data if desired.
def generateOptimizerRow(category, attr, pointEstimate, rows, isCategoryRow = False):
  categoryWidget = widgets.Label(category if isCategoryRow else "", layout=tableLayout)
  nameWidget = widgets.Label("" if isCategoryRow else stripQualifiers(attr), layout=tableLayout)
  min = 0
  max = 0
  if useTestData and not isCategoryRow:
    min = testData[stripQualifiers(attr)][0]
    max = testData[stripQualifiers(attr)][1]
  minWidget = widgets.IntText(value=min, layout=inputLayout)
  maxWidget = widgets.IntText(value=max, layout=inputLayout)

  minWidget.observe(attributeChangeHandler, names='value')
  maxWidget.observe(attributeChangeHandler, names='value')

  optSpendWidget = widgets.Output(layout=tableLayout)
  optRevWidget = widgets.Output(layout=tableLayout)
  warningWidget = widgets.Output(layout=warningLayout)
  rows.append({
      'name': nameWidget,
      'minSpend': minWidget,
      'maxSpend': maxWidget,
      'optSpend': optSpendWidget,
      'optRev': optRevWidget,
      'warning': warningWidget,
      'category': category,
      'attr': attr,
      'pointEstimate': pointEstimate,
      'isCategoryRow': isCategoryRow,
      'errors': set(),
      'displayRow': widgets.HBox([categoryWidget, nameWidget, minWidget, maxWidget, optSpendWidget, optRevWidget, warningWidget])
       })

def runp20Click(b):
  total = 0
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow'] and attr['category'] is not 'Total':
      total = total + original_data[attr['attr']].sum()

  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow'] and attr['category'] is not 'Total':
      rowTotal = original_data[attr['attr']].sum()
      attr['maxSpend'].value = (rowTotal/total) * metrics['budget'].value * 1.2
      attr['minSpend'].value = (rowTotal/total) * metrics['budget'].value * 0.8
  return

#Execution function for the actual optimizer. Fires off the minimization functions and handles all metric output.
def runOptimizerClick(b):
  metrics['loader'].add_class('lds-dual-ring')
  mins = []
  maxes = []
  orderedPointEstimates = []
  chosen_columns = []
  optSpendTotal = 0
  optRevTotal = 0
  totalSpends = []
  global final_iteration_revenue_data
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        mins.append(attr['minSpend'].value)
        maxes.append(attr['maxSpend'].value)
        orderedPointEstimates.append(attr['pointEstimate'])
        for col in feature_selected_transformed_w_target:
          if stripQualifiers(col, False) == attr['attr']:
            chosen_columns.append(col)
        totalSpends.append(original_data[attr['attr']].sum())
      else:
        printToOutputWidget(attr['minSpend'], sum(mins))
        printToOutputWidget(attr['maxSpend'], sum(maxes))
  
  #Actual optimization run here
  res = optimize(original_data, chosen_columns, orderedPointEstimates, mins, maxes, metrics['budget'].value)
  roiOpt = res.fun*-1
  printToOutputWidget(metrics['roiWidget'], round(roiOpt, ndigits=5))
  
  #Metric calculations for pRoi, originalRoi
  calculate_proi_original_data = original_data.drop(columns=[col for col in original_data if col not in [stripQualifiers(i, False) for i in chosen_columns]])
  total_original_spend = calculate_proi_original_data.sum().to_frame().sum()
  total_original_revenue = 0
  for col in list(feature_selected_transformed_w_o_target):
    category = categoryMapping[stripQualifiers(col, False)]['dropdown'].value
    if category != TARGET_VARIABLE and category != 'IGNORE' and category != "Non-Paid Media":
      total_original_revenue = total_original_revenue + (impByDriver['pointEstimate'][stripQualifiers(col, False)] * feature_selected_transformed_w_o_target[col].sum())
  original_roi = total_original_revenue/total_original_spend
  printToOutputWidget(metrics['originalRoiWidget'], round(original_roi[0], ndigits=5))
  pRoi = ((roiOpt - original_roi)/original_roi) * 100
  printToOutputWidget(metrics['pRoiWidget'], str(round(pRoi[0], ndigits=2)) + '%')
  
  #Metric calculations for optimized spend, optimized revenue, and spend reallocated (not currently displayed)
  if(pRoi.any() > 0):
    printToOutputWidget(metrics['status'], res.message)
  else:
    printToOutputWidget(metrics['status'], "This optimization maximized ROI subject to the provided budget and tactic level constraints.")
  attributeIterator = 0
  optimizedRevenueBreakdown = []
  totalSpendReallocated = 0
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        printToOutputWidget(attr['optSpend'], round(res.x[attributeIterator], ndigits=2))
        printToOutputWidget(attr['optRev'], round(final_iteration_revenue_data[attributeIterator], ndigits=2))
        if not np.isclose([res.x[attributeIterator]], [0.0]):
          totalSpendReallocated = totalSpendReallocated + abs(res.x[attributeIterator]-totalSpends[attributeIterator])
        attributeIterator += 1
      else:
        printToOutputWidget(attr['optSpend'], round(sum(res.x), ndigits=2))
        printToOutputWidget(attr['optRev'], round(sum(final_iteration_revenue_data), ndigits=2))
        printToOutputWidget(metrics['spendReallocated'], round(totalSpendReallocated, ndigits=2))
  metrics['loader'].remove_class('lds-dual-ring')

#Helper function for printing to an output widget
def printToOutputWidget(widget, data):
  with widget:
    widget.clear_output()
    if isinstance(data, str):
      print(data)
    else:
      print(formatNumber(data))

#Setup function for creating the display metrics in the grid
def createMetrics():
  budgetLabel = widgets.Label("Budget", layout=headerLayoutNoBottomMargin)
  roiLabel = widgets.Label("Optimized ROI", layout=headerLayoutNoBottomMargin)
  originalRoiLabel = widgets.Label("Original ROI", layout=headerLayoutNoBottomMargin)
  percentRoiLabel = widgets.Label("% Increase in ROI", layout=headerLayoutNoBottomMargin)
  spendReallocatedLabel = widgets.Label("Spend Reallocated", layout=headerLayoutNoBottomMargin)
  statusMessageLabel = widgets.Label("Status", layout=headerLayoutNoBottomMargin)
  budget = widgets.IntText(value=0, layout=headerLayout)
  budget.observe(budgetChangeHandler, names='value')
  originalRoiWidget = widgets.Output(layout=widgets.Layout(width='300px', display="flex", justify_content="center", margin="0px 0px 0px 8px"))
  roiWidget = widgets.Output(layout=headerLayout)
  pRoiWidget = widgets.Output(layout=headerLayout)
  spendReallocatedWidget = widgets.Output(layout=headerLayout)
  loaderWidget = widgets.HTML(html_loader)
  statusWidget = widgets.Output(layout=headerLayout)

  metrics= {
      'header': widgets.HBox([budgetLabel, originalRoiLabel, roiLabel, percentRoiLabel, statusMessageLabel]),
      'budget': budget,
      'originalRoiWidget': originalRoiWidget,
      'roiWidget': roiWidget,
      'pRoiWidget': pRoiWidget,
      'spendReallocated': spendReallocatedWidget,
      'loader': loaderWidget,
      'status': statusWidget,
      'displayRow': widgets.HBox([widgets.VBox([budget, p20button]), originalRoiWidget, roiWidget, pRoiWidget, statusWidget])
  }
  return metrics   

#Setup function for creating the header labels
def createOptimizerHeader():
  nameLabel = widgets.Label("Attribute Name", layout=tableLayout)
  categoryLabel = widgets.Label("Category Name", layout=tableLayout)
  minLabel = widgets.Label("MIN Spend", layout=inputLayout)
  maxLabel = widgets.Label("MAX Spend", layout=inputLayout)
  optSpendLabel = widgets.Label("Optimized Spend", layout=tableLayout)
  optRevLabel = widgets.Label("Optimized Revenue", layout=tableLayout)
  warningLabel = widgets.Label("Warnings", layout=warningLayout)
  return widgets.HBox([categoryLabel, nameLabel, minLabel, maxLabel, optSpendLabel, optRevLabel, warningLabel])

#Setup function for creating the footers
def createOptimizerFooter():
  totalLabel = widgets.Label("Total", layout=tableLayout)
  spacerWidget = widgets.Label("", layout=tableLayout)
  minWidget = widgets.Output(layout=inputLayout)
  maxWidget = widgets.Output(layout=inputLayout)
  optSpendWidget = widgets.Output(layout=widgets.Layout(width='300px', display="flex", justify_content="center", margin="0px 0px 0px 8px"))
  optRevWidget = widgets.Output(layout=tableLayout)
  warningWidget = widgets.Output(layout=warningLayout)
  rows.append({
      'name': totalLabel,
      'minSpend': minWidget,
      'maxSpend': maxWidget,
      'optSpend': optSpendWidget,
      'optRev': optRevWidget,
      'isCategoryRow': False,
      'category': 'Total',
      'displayRow': widgets.HBox([totalLabel, spacerWidget, minWidget, maxWidget, optSpendWidget, optRevWidget, warningWidget])
       })

#Setup function for creating the optimizer grid
def populateRows():
  invertCategoryMapping = {}
  for col in categoryMapping.keys():
    if categoryMapping[col]['dropdown'].value not in invertCategoryMapping.keys():
      invertCategoryMapping[categoryMapping[col]['dropdown'].value] = [col]
    else:
      invertCategoryMapping[categoryMapping[col]['dropdown'].value].append(col)
  for category in invertCategoryMapping.keys():
    if category != TARGET_VARIABLE and category != 'IGNORE' and category != "Non-Paid Media":
      generateOptimizerRow(category = category, attr = "", pointEstimate = 0, rows = rows, isCategoryRow = True)
      for attr in invertCategoryMapping[category]:
        generateOptimizerRow(category = category, attr = attr, pointEstimate = impByDriver['pointEstimate'][attr], rows = rows)

#Setup function for coordinating the entire grid
def populateDisplayUI(displayUIList):
  global metrics
  metrics = createMetrics()
  #displayUIList.append(widgets.HTML(html_styles))
  displayUIList.append(metrics['header'])
  displayUIList.append(metrics['displayRow'])
  displayUIList.append(createOptimizerHeader())
  populateRows()
  createOptimizerFooter()
  for row in rows:
    displayUIList.append(row['displayRow'])
  button.on_click(runOptimizerClick)
  p20button.on_click(runp20Click)
  displayUIList.append(widgets.HBox([button, metrics['loader']])) 

#Create grid and display it
populateDisplayUI(displayUI)
vbox = widgets.VBox(displayUI)
vbox.add_class('overflow_style')
widgets.VBox([widgets.HTML(html_styles), vbox])